In [2]:
import os
import networkx as nx
import matplotlib.pyplot as plt
import cython
import numpy as np
import pandas as pd
import datetime
import pylab

In [3]:
# load in merged data
data_path = '../../../UCB_dept_merge.csv'
mergeData = pd.read_csv(data_path)

In [4]:
# add in proper department names
data_path1 = '../../../Department_label.csv'
department = pd.read_csv(data_path1)

In [12]:
#clean up the department dataset
def convert_strings_to_specials(s):
    s = s.replace(' ', '_')
    s = s.replace(':', '_')
    s = s.replace('#', 'num')
    s = s.lower()
    return s
department.columns = [convert_strings_to_specials(col) for col in department.columns]

In [16]:
#change po_number to po_num for merge
department.rename(columns={'po_number':'po_num'}, inplace = True)

,﻿tier_3_-_uc_category_group,po_date,po_num,po_commitment,po_line_number,org_level_1,org_level_1_number,org_level_2_number,org_level_2,org_level_3,org_level_3_number,org_level_4,org_level_4_number,org_level_5,org_level_5_number,org_level_6,org_level_6_number,org_level_7,org_level_7_number
0,NaN,NaN,NaN,"1,378,222,902.63",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Building Construction,07/05/2012,BB00040468,300.00,1,UC Berkeley,UCBKL,CAMSU,Campus Support,Facilities Services,VCCPD,Capital Projects Operations,ALPDC,CP Operations,ALOPS,CP Operations-L6,ALOP6,Null from Data Source,Null from Data Source
2,Building Construction,07/16/2012,BB00044826,"2,500.00",1,UC Berkeley,UCBKL,CAMSU,Campus Support,Administration & Finance,VCBAS,Business & Administrative Svcs,BSAVC,University Police,FUPOL,UCPD Operations,FUOPS,Null from Data Source,Null from Data Source
3,Building Construction,07/20/2012,BB00047719,40.00,3,UC Berkeley,UCBKL,OACAD,Other Academic,University Extension,UNEX3,UNEX Administrative Depts,EXADM,Deans's Office Group,EXDOG,Deans's Office Group-L6,EXDO6,Null from Data Source,Null from Data Source
4,Building Construction,07/23/2012,BB00048237,"23,330.00",2,UC Berkeley,UCBKL,COLLE,Colleges,College of Engineering,COENG,Eng Dean's Office,EDDNO,Col of Engin Dean's Office,ED1DO,Col of Engin Dean's Office-L6,ED1D6,Null from Data Source,Null from Data Source


In [17]:
#merge the two dataframes by po_num
merged = pd.merge(mergeData, department, left_on='po_num', right_on='po_num', how='inner')

In [29]:
merged.head()

,po_id,po_num,creation_date,supplier_name,item_type,product_description,manufacturer,quantity,unit_price,department,...,org_level_3,org_level_3_number,org_level_4,org_level_4_number,org_level_5,org_level_5_number,org_level_6,org_level_6_number,org_level_7,org_level_7_number
0,29847796,BB00195884,2013-05-31,BELLCO GLASS INC,NonCatalog Product,"Septum Stopper, 20mm Blue Butyl Rubber QtyPerC...",NaN,1,77.41,NaN,...,College of Chemistry,COCHM,Dept Of Chemistry,CCHEM,Chemistry Dept,CCCHE,Chemistry Dept-L6,CCCH6,Null from Data Source,Null from Data Source
1,29847820,BB00195885,2013-05-31,CHEMGLASS LIFE SCIENCES LLC,NonCatalog Product,"Column, Chromatography, 24/40 Outer Joint, 100...",NaN,1,108.00,NaN,...,College of Chemistry,COCHM,Dept Of Chemistry,CCHEM,Chemistry Dept,CCCHE,Chemistry Dept-L6,CCCH6,Null from Data Source,Null from Data Source
2,29847565,BB00195881,2013-05-31,FISHER SCIENTIFIC,SQ Hosted Product,"Bottles, Media/Lab; Wheaton;Graduated; With ru...",Wheaton Science Products Inc,1,135.38,NaN,...,L&S Biological Sciences,LS1BS,Molecular & Cell Biology,IMMCB,MCB Research,IMRES,MCB Research-L6,IMRE6,Null from Data Source,Null from Data Source
3,29847565,BB00195881,2013-05-31,FISHER SCIENTIFIC,SQ Hosted Product,"Bottles, Media/Lab; Wheaton;Graduated; With ru...",Wheaton Science Products Inc,1,135.38,NaN,...,L&S Biological Sciences,LS1BS,Molecular & Cell Biology,IMMCB,MCB Research,IMRES,MCB Research-L6,IMRE6,Null from Data Source,Null from Data Source
4,29847565,BB00195881,2013-05-31,FISHER SCIENTIFIC,SQ Hosted Product,"Bottles, Media/Lab; Wheaton;Graduated; With ru...",Wheaton Science Products Inc,1,135.38,NaN,...,L&S Biological Sciences,LS1BS,Molecular & Cell Biology,IMMCB,MCB Research,IMRES,MCB Research-L6,IMRE6,Null from Data Source,Null from Data Source


In [27]:
#convert creation_date to datetime object
merged['creation_date']=pd.to_datetime(pd.Series(mergeData['creation_date']))
merged.columns = [convert_strings_to_specials(col) for col in merged.columns]
merged = merged[np.isfinite(merged['department_name'] 
merged.to_csv("merged.csv")

In [69]:
len(set(merged['org_level_5']))

586

In [44]:
firstDate = min(merged['creation_date'])
lastDate = max(merged['creation_date'])
timeframe = lastDate-firstDate
subsection = timeframe/12

In [72]:
firstDate

Timestamp('2012-01-18 00:00:00')

In [60]:
#function to give a bitartite graph and centrality measure of department and supplier by time
def testCentralityGraph(data, set1, set2, time_column, subsections = 12):
    results = []
    centrality = []
    nodes = []
    subset = data[[set1,set2,time_column]]
    firstDate = min(data[time_column])
    lastDate = max(data[time_column])
    timeframe = lastDate-firstDate
    partition = timeframe/subsections
    for i in range(subsections):
        current = subset[(subset[time_column]).isin(pd.date_range(firstDate + partition*i,firstDate+partition*(i+1)))]
        current = current[[set1,set2]]
        nodes += [current]
        current.dropna()
        G = nx.Graph()
        G.add_nodes_from(current[set1], bipartite = 0)
        G.add_nodes_from(current[set2], bipartite = 1)
        edgeList = [tuple(x) for x in current.values]
        G.add_edges_from(edgeList)
        betweenness = nx.betweenness_centrality(G, normalized=False, 
                                            weight=None)
        results += [(G, betweenness, (firstDate+timeframe*i,firstDate+timeframe*(i+1)))]
        centrality+= [betweenness]
    
    return results, centrality,nodes

In [61]:
test,centrality, n = testCentralityGraph(merged,'org_level_4', 'supplier_name','creation_date')

In [62]:
print n

[                        org_level_4                supplier_name
271042  Capital Projects Operations  ONE WORKPLACE L FERRARI LLC
271043  Capital Projects Operations  ONE WORKPLACE L FERRARI LLC
271044  Capital Projects Operations  ONE WORKPLACE L FERRARI LLC
271045  Capital Projects Operations  ONE WORKPLACE L FERRARI LLC
271046  Capital Projects Operations  ONE WORKPLACE L FERRARI LLC
271047  Capital Projects Operations  ONE WORKPLACE L FERRARI LLC
271048  Capital Projects Operations  ONE WORKPLACE L FERRARI LLC
271049  Capital Projects Operations  ONE WORKPLACE L FERRARI LLC
271050  Capital Projects Operations  ONE WORKPLACE L FERRARI LLC
271051  Capital Projects Operations  ONE WORKPLACE L FERRARI LLC
271052  Capital Projects Operations  ONE WORKPLACE L FERRARI LLC
271053  Capital Projects Operations  ONE WORKPLACE L FERRARI LLC
271054  Capital Projects Operations  ONE WORKPLACE L FERRARI LLC
271055  Capital Projects Operations  ONE WORKPLACE L FERRARI LLC
271056  Capital Projects

In [56]:
#subset by date
first3Months = deptSupplier[deptSupplier['creation_date'] < datetime.datetime(2012,3,1)]
first3Months = first3Months.dropna() 
len(first3Months)

636

In [57]:
#keep only the unique values
wanted = first3Months[['department_name','supplier_name']]
wanted.drop_duplicates()

,department_name,supplier_name
273362,MDNST,SANTA CRUZ BIOTECHNOLOGY INC
273361,MDNST,OPEN BIOSYSTEMS INC
273360,MEPMB,FISHER SCIENTIFIC
273354,MDNST,FISHER SCIENTIFIC
273341,ALPDC,GLS ARCHITECT LANDSCAPE ARCHITECT
273342,EERES,VWR INTERNATIONAL INC
273348,ALPDC,VILA CONSTRUCTION COMPANY
273349,MEPMB,APPLE COMPUTER INC
273351,ALPDC,Carl Zeiss Microscopy LLC
273332,EERES,ALKO OFFICE SUPPLY


In [58]:
#make a graph
test= nx.Graph()
edges = [tuple(x) for x in wanted.values]
test.add_edges_from(edges)
#position = {department:0 for department in wanted["department_name"]}
#for supplier in wanted["supplier_name"]: position[supplier] = 1
values = range(0,len(test.nodes()))
#len(position) ==len(values)

In [ ]:
nx.draw(test, node_color = values,with_labels = True)
plt.show()
plt.savefig("test.png")